In [68]:
%%time
import re
import pandas as pd

Wall time: 0 ns


In [69]:
%%time
import glob
raw_data_files = ['access_log_Jul95','access_log_Aug95']

Wall time: 0 ns


In [70]:
%%time
df = pd.DataFrame()
for ijk in raw_data_files:
    frame = pd.read_csv(ijk,encoding='unicode_escape', error_bad_lines=False,escapechar=",", header=None)
    df = df.append(frame)
df.columns = ['text']

Wall time: 16 s


In [71]:
%%time
def make_host_col(val):
    host_pattern = r'(^\S+\.[\S+\.]+\S+)\s' 
    z = re.search(host_pattern, val)
    return z.group(1) if z else None
def make_timestamp_col(val):
    ts_pattern = r'\[(\d{2}/\w{3}/\d{4}:\d{2}:\d{2}:\d{2} -\d{4})]'
    z = re.search(ts_pattern, val)
    return z.group(1) if z else None
def make_http_codes_col(val):
    content_size_pattern = r'\s(\d+)$'
    z = re.search(content_size_pattern, val)
    return z.group(1) if z else None    
def make_http_codes_col(val):
    status_pattern = r'\s(\d{3})\s'
    z = re.search(status_pattern, val)
    return z.group(1) if z else None
def make_url_req_protocol_col(val):
    method_uri_protocol_pattern = r'\"(\S+)\s(\S+)\s*(\S*)\"'
    z = re.search(method_uri_protocol_pattern, val)
    return z.groups() if z else (None,None,None)

Wall time: 0 ns


In [72]:
%%time
df['timestamp'] = df['text'].apply(make_timestamp_col)
df['hosts'] = df['text'].apply(make_host_col) 
df['content_size'] = df['text'].apply(make_http_codes_col)
df['status'] = df['text'].apply(make_http_codes_col)
df['test'] = df['text'].apply(make_url_req_protocol_col)
df['method'],df['endpoint'],df['protocol'] = df['test'].str[0],df['test'].str[1],df['test'].str[2]

Wall time: 57.5 s


In [73]:
%%time
df.drop(['test','text'],axis=1,inplace=True)

Wall time: 2.16 s


In [74]:
df.head(20)

,timestamp,hosts,content_size,status,method,endpoint,protocol
0,01/Jul/1995:00:00:01 -0400,199.72.81.55,200,200,GET,/history/apollo/,HTTP/1.0
1,01/Jul/1995:00:00:06 -0400,unicomp6.unicomp.net,200,200,GET,/shuttle/countdown/,HTTP/1.0
2,01/Jul/1995:00:00:09 -0400,199.120.110.21,200,200,GET,/shuttle/missions/sts-73/mission-sts-73.html,HTTP/1.0
3,01/Jul/1995:00:00:11 -0400,burger.letters.com,304,304,GET,/shuttle/countdown/liftoff.html,HTTP/1.0
4,01/Jul/1995:00:00:11 -0400,199.120.110.21,200,200,GET,/shuttle/missions/sts-73/sts-73-patch-small.gif,HTTP/1.0
5,01/Jul/1995:00:00:12 -0400,burger.letters.com,304,304,GET,/images/NASA-logosmall.gif,HTTP/1.0
6,01/Jul/1995:00:00:12 -0400,burger.letters.com,200,200,GET,/shuttle/countdown/video/livevideo.gif,HTTP/1.0
7,01/Jul/1995:00:00:12 -0400,205.212.115.106,200,200,GET,/shuttle/countdown/countdown.html,HTTP/1.0
8,01/Jul/1995:00:00:13 -0400,d104.aa.net,200,200,GET,/shuttle/countdown/,HTTP/1.0
9,01/Jul/1995:00:00:13 -0400,129.94.144.152,200,200,GET,/,HTTP/1.0


In [75]:
f1 = df['timestamp']=="" 
f2 = df['hosts']==None
f3 = df['content_size']==None
f4 = df['status']==None
f5 = df['method']==None
f6 = df['endpoint']==None
f7 = df['protocol']==None
# df[f1|f2|f3|f4|f5|f6|f7]
df[f1]

,timestamp,hosts,content_size,status,method,endpoint,protocol


In [76]:
month_map = {
  'Jan': 1, 'Feb': 2, 'Mar':3, 'Apr':4, 'May':5, 'Jun':6, 'Jul':7,
  'Aug':8,  'Sep': 9, 'Oct':10, 'Nov': 11, 'Dec': 12
}

def parse_clf_time(text):
    return "{0:04d}-{1:02d}-{2:02d} {3:02d}:{4:02d}:{5:02d}".format(
      int(text[7:11]),
      month_map[text[3:6]],
      int(text[0:2]),
      int(text[12:14]),
      int(text[15:17]),
      int(text[18:20])
    ) if text is not None else None
df['timestamp']=df['timestamp'].apply(parse_clf_time)
df.head(20)

,timestamp,hosts,content_size,status,method,endpoint,protocol
0,1995-07-01 00:00:01,199.72.81.55,200,200,GET,/history/apollo/,HTTP/1.0
1,1995-07-01 00:00:06,unicomp6.unicomp.net,200,200,GET,/shuttle/countdown/,HTTP/1.0
2,1995-07-01 00:00:09,199.120.110.21,200,200,GET,/shuttle/missions/sts-73/mission-sts-73.html,HTTP/1.0
3,1995-07-01 00:00:11,burger.letters.com,304,304,GET,/shuttle/countdown/liftoff.html,HTTP/1.0
4,1995-07-01 00:00:11,199.120.110.21,200,200,GET,/shuttle/missions/sts-73/sts-73-patch-small.gif,HTTP/1.0
5,1995-07-01 00:00:12,burger.letters.com,304,304,GET,/images/NASA-logosmall.gif,HTTP/1.0
6,1995-07-01 00:00:12,burger.letters.com,200,200,GET,/shuttle/countdown/video/livevideo.gif,HTTP/1.0
7,1995-07-01 00:00:12,205.212.115.106,200,200,GET,/shuttle/countdown/countdown.html,HTTP/1.0
8,1995-07-01 00:00:13,d104.aa.net,200,200,GET,/shuttle/countdown/,HTTP/1.0
9,1995-07-01 00:00:13,129.94.144.152,200,200,GET,/,HTTP/1.0


In [77]:
df.to_csv('nasa_final_log.csv')